FEED-FORWARD NEURAL NETWORK

In [ ]:

import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

#step1...split_data
X, y = load_breast_cancer(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#step2...Scale features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

#step3...convert_to_torch_tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
y_test = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)

#step4...create feedforward_NN, {activation_factors- ReLU and Sigmoid}
class FeedForwardNN(nn.Module):    #create_class_nd_make_layered_neurons
    def __init__(self, input_dim):
        super(FeedForwardNN, self).__init__()
        self.fc1 = nn.Linear(input_dim, 32)  # Input -> Hidden Layer (32 neurons)
        self.fc2 = nn.Linear(32, 16)         # Hidden -> Hidden (16 neurons)
        self.fc3 = nn.Linear(16, 1)          # Hidden -> Output (1 neuron for binary classification)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.sigmoid(self.fc3(x))  # Output probability
        return x
model = FeedForwardNN(X_train.shape[1])

#step5... optimizer$loss
criterion = nn.BCELoss()  # Binary Cross Entropy for binary classification
optimizer = optim.Adam(model.parameters(), lr=0.001)  #adam_optimizer

#step6....train_the_model
epochs = 50
for epoch in range(epochs):
    # Forward pass
    outputs = model(X_train)
    loss = criterion(outputs, y_train)

    # Backward pass
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch+1) % 10 == 0:
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}")

#step6....evaluate_the_model
with torch.no_grad():
    y_pred = model(X_test)
    y_pred_cls = (y_pred >= 0.5).float()
    acc = accuracy_score(y_test, y_pred_cls)
    print(f"Feedforward NN Accuracy: {acc:.4f}")


Epoch [10/50], Loss: 0.5865
Epoch [20/50], Loss: 0.5137
Epoch [30/50], Loss: 0.4365
Epoch [40/50], Loss: 0.3588
Epoch [50/50], Loss: 0.2883
Feedforward NN Accuracy: 0.9737


In [ ]:
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

# Step1.....Load MNIST dataset
transform = transforms.Compose([transforms.ToTensor()])
train_dataset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

#step2...develop_the_CNN_Architecture
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, padding=1) # 1 input channel, 16 filters
        self.pool = nn.MaxPool2d(2, 2)                         # 2x2 pooling
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(32*7*7, 128)                      # Fully connected
        self.fc2 = nn.Linear(128, 10)                          # Output 10 classes (digits)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.pool(x)
        x = self.relu(self.conv2(x))
        x = self.pool(x)
        x = x.view(-1, 32*7*7)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

model = CNN()

# step3....Loss & Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# step4....Train
for epoch in range(1):
    for images, labels in train_loader:
        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f"Epoch [{epoch+1}], Loss: {loss.item():.4f}")

# step5..... Evaluate_the_model
correct, total = 0, 0
with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"CNN Accuracy: {100*correct/total:.2f}%")


100%|██████████| 9.91M/9.91M [00:00<00:00, 59.5MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 1.73MB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 14.3MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 9.45MB/s]


Epoch [1], Loss: 0.1535
CNN Accuracy: 97.93%


RECURRENT NEURAL NETWORK

In [ ]:

import numpy as np

# step1......Generate sine wave data
x = np.linspace(0, 100, 500)
y = np.sin(x)

#step2....Prepare sequences
def create_sequences(data, seq_length=10):
    xs, ys = [], []
    for i in range(len(data)-seq_length):
        xs.append(data[i:i+seq_length])
        ys.append(data[i+seq_length])
    return np.array(xs), np.array(ys)

seq_length = 10
X_seq, y_seq = create_sequences(y, seq_length)
X_seq = torch.tensor(X_seq, dtype=torch.float32).unsqueeze(-1) # shape (samples, seq_len, 1)
y_seq = torch.tensor(y_seq, dtype=torch.float32).view(-1, 1)

#ste3...build_rnn_archi
class RNNModel(nn.Module):
    def __init__(self, input_size=1, hidden_size=32, num_layers=1):
        super(RNNModel, self).__init__()
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, 1)

    def forward(self, x):
        out, _ = self.rnn(x)
        out = self.fc(out[:, -1, :]) # last time step
        return out
#step4....optimizer&loss
model = RNNModel()
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

#step5...train&evaluate
for epoch in range(100):
    outputs = model(X_seq)
    loss = criterion(outputs, y_seq)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch+1) % 20 == 0:
        print(f"Epoch [{epoch+1}/100], Loss: {loss.item():.4f}")



Epoch [20/100], Loss: 0.0047
Epoch [40/100], Loss: 0.0004
Epoch [60/100], Loss: 0.0002
Epoch [80/100], Loss: 0.0001
Epoch [100/100], Loss: 0.0001
